In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# All Imports

In [ ]:
import numpy as np
import scipy
import scipy.stats
import h5py
from numpy.random import RandomState
import IPython
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
from PIL import Image

import os
import shutil
import math

import tensorflow as tf
import keras
from keras.models import Sequential
from keras import layers
from keras.layers import Conv2D, Dropout, Flatten, Dense, Activation, ZeroPadding2D, LeakyReLU, BatchNormalization
from keras.layers import Input, Convolution1D, Concatenate, UpSampling2D, Conv2DTranspose
from keras import applications
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.models import Model
from keras.utils import to_categorical
from keras.activations import relu, softmax
from keras import losses, models, optimizers
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adadelta, RMSprop,SGD,Adam

# Pytorch
import torch
import torch.nn as nn
import torch.nn.init as init
from torch import nn, optim
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.utils import data
import pickle

#Pre-Processing Functions

In [3]:
def zero_pad(X, pad):
  # input X - python numpy array of shape (m, n_H, n_W) representing a batch of m images
  # input pad - integer
  # Output X_pad - padded image of shape (m, n_H + 2*pad, n_W + 2*pad)

    X_pad = np.zeros([X.shape[0], X.shape[1]+2*pad, X.shape[2]+2*pad])
    for i in range(X.shape[0]):
      X_pad[i] = np.pad(X[i], pad, mode='constant')
    
    return X_pad

def zero_pad_3d(X, pad):
  # input X - python numpy array of shape (m, n_H, n_W,3) representing a batch of m images
  # input pad - integer
  # Output X_pad - padded image of shape (m, n_H + 2*pad, n_W + 2*pad,3)

    X_pad = np.zeros([X.shape[0], X.shape[1]+2*pad, X.shape[2]+2*pad, X.shape[3]])
    for i in range(X.shape[0]):
      X_pad[i] = np.pad(X[i], ((pad,pad), (pad,pad), (0,0)), mode='constant')
    
    return X_pad

In [4]:
def get_total_cnt_no_redundancy(L):
  # L is the image having just point annotations
  count = np.count_nonzero(L>1)
  return count

In [5]:
def get_total_cnt_with_redundancy(T, f=32, stride=1):
  # T is the image having redundant counts
  # f is the size of the receptive field (the 'r' in the paper)
  count = np.sum(T)
  count = count/(f*f)
  count = count*(stride*stride)
  return count

In [6]:
def conv_LtoT(L, stride=1, pad=32, f=32):
    """
    Arguments:
    L -- numpy array of m images
    stride -- stride used
    pad -- padding for L
    f -- The receptive field size ('r' in the paper)
        
    Returns:
    Z -- conversion output, numpy array of shape (m, n_H, n_W)
    """
     
    (m, n_H_prev, n_W_prev) = (L.shape[0] , L.shape[1] , L.shape[2])
    n_H = int( (n_H_prev - f + 2*pad)/stride ) 
    n_W = int( (n_W_prev - f + 2*pad)/stride )
    
    Z = np.zeros([m,n_H,n_W])
    L_pad = zero_pad(L,pad)
    
    for i in range(m):                               # loop over the batch of training examples
        a_pad = L_pad[i]                               # Select ith training example's padded activation
        for h in range(n_H):                           # loop over vertical axis of the output volume
            for w in range(n_W):                       # loop over horizontal axis of the output volume
               
                    # Find the corners of the current "slice"
                    vert_start = h*stride
                    vert_end = h*stride + f
                    horiz_start = w*stride
                    horiz_end = w*stride + f
                    
                    a_slice = a_pad[vert_start:vert_end,horiz_start:horiz_end]
                    
                    Z[i, h, w] = get_total_cnt_no_redundancy(a_slice)
                         
    
    # Making sure your output shape is correct
    assert(Z.shape == (m, n_H, n_W))
    return Z

# Gaussian

In [7]:
def genGausImage(framesize, mx, my, cov=1):
    x, y = np.mgrid[0:framesize, 0:framesize]
    pos = np.dstack((x, y))
    mean = [mx, my]
    cov = [[cov, 0], [0, cov]]
    rv = scipy.stats.multivariate_normal(mean, cov).pdf(pos)
    # print(rv.shape)
    return rv/rv.sum()

In [8]:
def getDensity(height, width, markers, f=32, cov=1):
    gaus_img = np.zeros((height,width))
    # print(gaus_img.shape)
    for k in range(height):
        for l in range(width):
            if (markers[k,l] > 1):
                gaus_img += genGausImage(len(markers),k-f/2,l-f/2,cov)
    return gaus_img

In [9]:
def conv_LtoT_gaussian(L, stride=1, pad=32, f=32):
    """
    Arguments:
    L -- numpy array of m images
    stride -- stride used
    pad -- padding for L
    f -- The receptive field size ('r' in the paper)
        
    Returns:
    Z -- conversion output, numpy array of shape (m, n_H, n_W)
    """
      
    (m, n_H_prev, n_W_prev) = (L.shape[0] , L.shape[1] , L.shape[2])
    n_H = int( (n_H_prev - f + 2*pad)/stride ) 
    n_W = int( (n_W_prev - f + 2*pad)/stride )
    
    T = np.zeros([m,n_H,n_W])
    L_pad = zero_pad(L,pad)
    
    for i in range(m):                               # loop over the batch of training examples
        L_modified = L_pad[i][0:n_H, 0:n_W]
        T[i] = getDensity(n_H, n_W, L_modified, f)
                                        
    
    # Making sure your output shape is correct
    assert(T.shape == (m, n_H, n_W))
    return T

In [10]:
# Pre-Processing Parameters:

receptive_field = 32
stride = 1
cells_kernel = 'square'
adipocyte_kernel = 'square'
mbm_kernel = 'square'


#CELLS DATA

In [11]:
!cp -r "drive/My Drive/countception-master/cells" "cells"

In [12]:
I_data_cells = []
L_data_cells = []
count_data_cells = []

path = 'cells/'

for i in range(1,201):
  s_I = path + f'{i:03}' + 'cell.png'
  s_L = path + f'{i:03}' + 'dots.png' 

  I = cv2.imread(s_I)
  I = I.transpose((2,0,1))
  I_data_cells.append(I)

  L = cv2.imread(s_L, 0)
  L_data_cells.append(L)

  count = get_total_cnt_no_redundancy(L)
  count_data_cells.append(count)

I_data_cells = np.array(I_data_cells)
L_data_cells = np.array(L_data_cells)
count_data_cells = np.array(count_data_cells)

if cells_kernel == 'square':
  T_data_cells = conv_LtoT(L_data_cells, stride=stride, f=receptive_field)
else:
  T_data_cells = conv_LtoT_gaussian(L_data_cells, stride=stride, f=receptive_field)
T_data_cells = T_data_cells.reshape(T_data_cells.shape[0], 1, T_data_cells.shape[1], T_data_cells.shape[2])


In [ ]:
print(I_data_cells.shape)
print(L_data_cells.shape)
print(T_data_cells.shape)
print(count_data_cells.shape)


# ADIPOCYTE DATA

In [48]:
!cp -r "drive/My Drive/countception-master/adipocyte_data" "adipocyte_data"

In [49]:
I_data_adipocyte = []
L_data_adipocyte = []
count_data_adipocyte = []

path_I = 'adipocyte_data/images/'
path_L = 'adipocyte_data/annotations/'
filenames = []

for filename in os.listdir(path_I):
  filenames.append(filename)

for filename in filenames:
  s_I = path_I + filename
  s_L = path_L + filename 

  I = cv2.imread(s_I)
  I = I.transpose((2,0,1))
  I_data_adipocyte.append(I)

  L = cv2.imread(s_L, 0)
  L_data_adipocyte.append(L)

  count = get_total_cnt_no_redundancy(L)
  count_data_adipocyte.append(count)

I_data_adipocyte = np.array(I_data_adipocyte)
L_data_adipocyte = np.array(L_data_adipocyte)
count_data_adipocyte = np.array(count_data_adipocyte)

if adipocyte_kernel == 'square':
  T_data_adipocyte = conv_LtoT(L_data_adipocyte, stride=stride, f=receptive_field)
else:
  T_data_adipocyte = conv_LtoT_gaussian(L_data_adipocyte, stride=stride, f=receptive_field)
T_data_adipocyte = T_data_adipocyte.reshape(T_data_adipocyte.shape[0], 1, T_data_adipocyte.shape[1], T_data_adipocyte.shape[2])

In [ ]:
print(I_data_adipocyte.shape)
print(L_data_adipocyte.shape)
print(T_data_adipocyte.shape)
print(count_data_adipocyte.shape)

#MBM DATA

In [ ]:
!cp -r "drive/My Drive/countception-master/MBM_data" "MBM_data"

In [ ]:
I_data_mbm = []
L_data_mbm = []
count_data_mbm = []

path = 'MBM_data/'
filenames_I = []
filenames_L = []

for filename in os.listdir(path):
  if 'dots' in filename:
    filenames_L.append(filename)
  else:
    filenames_I.append(filename)

filenames_I.sort()
filenames_L.sort()

for i in range(len(filenames_I)):
  s_I = path + filenames_I[i]
  s_L = path + filenames_L[i] 

  # I = Image.open(s_I)
  # # I = I.resize((300, 300))
  # I = np.array(I)
  # I = I.transpose((2,0,1))
  # I_data_mbm.append(I)

  I = cv2.imread(s_I)
  # I = np.array(Image.fromarray(I, mode='RGB').resize((300,300)))
  I = I.transpose((2,0,1))
  I_data_mbm.append(I)

  # L = Image.open(s_L).convert('L')
  # # L = L.resize((300, 300))
  # L = np.array(L)
  # L_data_mbm.append(L)

  L = cv2.imread(s_L, 0)
  # L = np.array(Image.fromarray(L).resize((300,300)))
  L_data_mbm.append(L)

  count = get_total_cnt_no_redundancy(L)
  count_data_mbm.append(count)

I_data_mbm = np.array(I_data_mbm)
L_data_mbm = np.array(L_data_mbm)
count_data_mbm = np.array(count_data_mbm)

if mbm_kernel == 'square':
  T_data_mbm = conv_LtoT(L_data_mbm, stride=stride, f=receptive_field)
else:
  T_data_mbm = conv_LtoT_gaussian(L_data_mbm, stride=stride, f=receptive_field)
T_data_mbm = T_data_mbm.reshape(T_data_mbm.shape[0], 1, T_data_mbm.shape[1], T_data_mbm.shape[2])

In [ ]:
print(I_data_mbm.shape)
print(L_data_mbm.shape)
print(T_data_mbm.shape)
print(count_data_mbm.shape)

# Train-Test-Val Split

In [14]:
def train_val_test_split(I_array, L_array, T_array, count_array, test_data_percent=0.60, val_data_percent=0.20):

  number_of_samples = I_array.shape[0]
  
  index1 = np.random.choice(number_of_samples, math.ceil(test_data_percent*number_of_samples), replace=False)
  mask1 = np.zeros(number_of_samples,dtype=bool)
  mask1[index1] = True

  temp = np.zeros(number_of_samples)
  temp = temp[~mask1]

  index2 = np.random.choice(temp.shape[0], math.ceil((val_data_percent/(1-test_data_percent))*temp.shape[0]), replace=False)
  mask2 = np.zeros(temp.shape[0],dtype=bool)
  mask2[index2] = True

  I_test = I_array[mask1]
  L_test = L_array[mask1]
  T_test = T_array[mask1]
  count_test = count_array[mask1]

  I_train_val = I_array[~mask1]
  L_train_val = L_array[~mask1]
  T_train_val = T_array[~mask1]
  count_train_val = count_array[~mask1]

  I_val = I_train_val[mask2]
  L_val = L_train_val[mask2]
  T_val = T_train_val[mask2]
  count_val = count_train_val[mask2]

  I_train = I_train_val[~mask2]
  L_train = L_train_val[~mask2]
  T_train = T_train_val[~mask2]
  count_train = count_train_val[~mask2]

  final_list = []
  final_list.append(I_train)
  final_list.append(I_val)
  final_list.append(I_test)
  final_list.append(L_train)
  final_list.append(L_val)
  final_list.append(L_test)
  final_list.append(T_train)
  final_list.append(T_val)
  final_list.append(T_test)
  final_list.append(count_train)
  final_list.append(count_val)
  final_list.append(count_test)

  return final_list

CELLS DATA

In [15]:
temp_cells = train_val_test_split(I_data_cells, L_data_cells, T_data_cells, count_data_cells, test_data_percent=0.60, val_data_percent=0.20)

I_train_cells = temp_cells[0]
I_val_cells = temp_cells[1]
I_test_cells = temp_cells[2]
L_train_cells = temp_cells[3]
L_val_cells = temp_cells[4]
L_test_cells = temp_cells[5]
T_train_cells = temp_cells[6]
T_val_cells = temp_cells[7]
T_test_cells = temp_cells[8]
count_train_cells = temp_cells[9]
count_val_cells = temp_cells[10]
count_test_cells = temp_cells[11]

In [ ]:
print(I_train_cells.shape, I_val_cells.shape, I_test_cells.shape)
print(L_train_cells.shape, L_val_cells.shape, L_test_cells.shape)
print(T_train_cells.shape, T_val_cells.shape, T_test_cells.shape)
print(count_train_cells.shape, count_val_cells.shape, count_test_cells.shape)

ADIPOCYTE DATA

In [52]:
temp_adipocyte = train_val_test_split(I_data_adipocyte, L_data_adipocyte, T_data_adipocyte, count_data_adipocyte, test_data_percent=0.60, val_data_percent=0.20)

I_train_adipocyte = temp_adipocyte[0]
I_val_adipocyte = temp_adipocyte[1]
I_test_adipocyte = temp_adipocyte[2]
L_train_adipocyte = temp_adipocyte[3]
L_val_adipocyte = temp_adipocyte[4]
L_test_adipocyte = temp_adipocyte[5]
T_train_adipocyte = temp_adipocyte[6]
T_val_adipocyte = temp_adipocyte[7]
T_test_adipocyte = temp_adipocyte[8]
count_train_adipocyte = temp_adipocyte[9]
count_val_adipocyte = temp_adipocyte[10]
count_test_adipocyte = temp_adipocyte[11]

In [ ]:
print(I_train_adipocyte.shape, I_val_adipocyte.shape, I_test_adipocyte.shape)
print(L_train_adipocyte.shape, L_val_adipocyte.shape, L_test_adipocyte.shape)
print(T_train_adipocyte.shape, T_val_adipocyte.shape, T_test_adipocyte.shape)
print(count_train_adipocyte.shape, count_val_adipocyte.shape, count_test_adipocyte.shape)

MBM DATA

In [ ]:
temp_mbm = train_val_test_split(I_data_mbm, L_data_mbm, T_data_mbm, count_data_mbm, test_data_percent=0.40, val_data_percent=0.11)

I_train_mbm = temp_mbm[0]
I_val_mbm = temp_mbm[1]
I_test_mbm = temp_mbm[2]
L_train_mbm = temp_mbm[3]
L_val_mbm = temp_mbm[4]

L_test_mbm = temp_mbm[5]
T_train_mbm = temp_mbm[6]
T_val_mbm = temp_mbm[7]
T_test_mbm = temp_mbm[8]
count_train_mbm = temp_mbm[9]
count_val_mbm = temp_mbm[10]
count_test_mbm = temp_mbm[11]

In [ ]:
print(I_train_mbm.shape, I_val_mbm.shape, I_test_mbm.shape)
print(L_train_mbm.shape, L_val_mbm.shape, L_test_mbm.shape)
print(T_train_mbm.shape, T_val_mbm.shape, T_test_mbm.shape)
print(count_train_mbm.shape, count_val_mbm.shape, count_test_mbm.shape)

# PyTorch Self-Implementation

In [17]:
class Main_Conv_Layer(nn.Module):
    def __init__(self, in_channel, out_channel, kernel_size=3, stride=1, pad=0, activation=nn.LeakyReLU(0.01)):
        super(Main_Conv_Layer, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channel, out_channels=out_channel, kernel_size=kernel_size, stride=stride, padding=pad)
        self.activation = activation
        self.batch_norm = nn.BatchNorm2d(out_channel)

    def forward(self, x):
        return self.activation(self.batch_norm(self.conv1(x)))


class Two_Conv(nn.Module):
    def __init__(self, in_channel, out_channel_1x1, out_channel_3x3, activation=nn.LeakyReLU(0.01)):
        super(Two_Conv, self).__init__()
        self.conv1 = Main_Conv_Layer(in_channel, out_channel_1x1, kernel_size=1, pad=0, activation=activation)
        self.conv2 = Main_Conv_Layer(in_channel, out_channel_3x3, kernel_size=3, pad=1, activation=activation)

    def forward(self, x):
        conv1_out = self.conv1(x)
        conv2_out = self.conv2(x)
        output = torch.cat([conv1_out, conv2_out], 1)
        return output


class ModelCountception(nn.Module):
    def __init__(self, in_channel=3, out_channel=1, initial_padding=32):
        super(ModelCountception, self).__init__()

        # parameters
        self.in_channel = in_channel
        self.out_channel = out_channel
        self.activation = nn.LeakyReLU(0.01)
        self.initial_padding = initial_padding

        torch.LongTensor()

        # layers (comment size for cells dataset)
        self.conv1 = Main_Conv_Layer(self.in_channel, 64, kernel_size=3, pad=self.initial_padding, activation=self.activation) #(64,318,318)
        self.conv2 = Two_Conv(64, 16, 16, activation=self.activation) #(32,318,318)
        self.conv3 = Two_Conv(32, 16, 32, activation=self.activation) #(48,318,318)
        self.conv4 = Main_Conv_Layer(48, 16, kernel_size=14, activation=self.activation) #(16,305,305)
        self.conv5 = Two_Conv(16, 112, 48, activation=self.activation) #(160,305,305)
        self.conv6 = Two_Conv(160, 64, 32, activation=self.activation) #(96,305,305)
        self.conv7 = Two_Conv(96, 40, 40, activation=self.activation) #(80,305,305)
        self.conv8 = Two_Conv(80, 32, 96, activation=self.activation) #(128,305,305)
        self.conv9 = Main_Conv_Layer(128, 32, kernel_size=18, activation=self.activation) #(32,288,288)
        self.conv10 = Main_Conv_Layer(32, 64, kernel_size=1, activation=self.activation) #(64,288,288)
        self.conv11 = Main_Conv_Layer(64, 64, kernel_size=1, activation=self.activation) #(64,288,288)
        self.conv12 = Main_Conv_Layer(64, self.out_channel, kernel_size=1, activation=self.activation) #(1,288,288)

        # Weight initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
                init.xavier_uniform(m.weight, gain=init.calculate_gain('leaky_relu', param=0.01))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()


    def forward(self, x):
        net = self.conv1(x)
        net = self.conv2(net)
        net = self.conv3(net)
        net = self.conv4(net)
        net = self.conv5(net)
        net = self.conv6(net)
        net = self.conv7(net)
        net = self.conv8(net)
        net = self.conv9(net)
        net = self.conv10(net)
        net = self.conv11(net)
        net = self.conv12(net)
        
        return net

    def name(self):
        return 'countception'

In [18]:
def get_output_count(x_train, y_train, x_val, y_val, x_test, y_test=None, is_print=True, batch_size=4, epochs=1000, lr=0.05, test_batch_size=1, data_type='cells', kernel_type='square'):
  if y_test is None:
    y_test = np.zeros(1)

  x_train, y_train, x_val, y_val, x_test, y_test = map(torch.tensor, (x_train, y_train, x_val, y_val, x_test, y_test))

  train_ds = TensorDataset(x_train, y_train)
  train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=8)

  val_ds = TensorDataset(x_val, y_val)
  val_dl = DataLoader(val_ds, batch_size=batch_size, num_workers=8)

  test_ds = TensorDataset(x_test, y_test)
  test_dl = DataLoader(test_ds, batch_size=test_batch_size, num_workers=8)

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  criterion = nn.L1Loss()
  model = ModelCountception().to(device)
  solver = optim.Adam(model.parameters(), lr=lr)

  for epoch in range(epochs):
    train_loss = []
    for idx, (inputt, target) in enumerate(train_dl):
        inputt = inputt.type(torch.FloatTensor)
        target = target.type(torch.FloatTensor)
        inputt = inputt.to(device)
        target = target.to(device)
        output = model.forward(inputt)
        loss = criterion(output, target)

        # Zero grad
        model.zero_grad()
        loss.backward()
        solver.step()

        train_loss.append(loss.data.cpu().numpy())
    if is_print:
        print("Epoch", epoch, "- Training Loss:", np.mean(train_loss), end=',  ')

    with torch.no_grad():
        val_loss = []
        for idx, (inputt, target) in enumerate(val_dl):
            inputt = inputt.type(torch.FloatTensor)
            target = target.type(torch.FloatTensor)
            inputt = inputt.to(device)
            target = target.to(device)
            output = model.forward(inputt)
            val_loss.append(criterion(output, target).data.cpu().numpy())
        if is_print:
            print("Epoch", epoch, "- Validation Loss:", np.mean(val_loss))

    if (epoch+1) % 5 == 0 or epoch == epochs-1:
        state = {'model_weights': model.state_dict()}
        torch.save(state, data_type + "after_{0}_epochs.model".format(epoch))

  criterion = nn.L1Loss()
  model = ModelCountception().to(device)
  model.eval()

  last = str(epochs-1)
  from_before = torch.load(data_type + 'after_' + last + '_epochs.model')
  model_weights = from_before['model_weights']
  model.load_state_dict(model_weights)


  predicted_T = []

  test_loss = []
  with torch.no_grad():
      for idx, (inputt, target) in enumerate(test_dl):
          inputt = inputt.type(torch.FloatTensor)
          target = target.type(torch.FloatTensor)
          inputt = inputt.to(device)
          target = target.to(device)
          output = model.forward(inputt)
          test_loss.append(criterion(output, target).data.cpu().numpy())

          output = output.cpu().numpy()
          for i in range(output.shape[0]):
            predicted_T.append(output[i])

      if is_print:
          print('MAE of Test Set: ', np.mean(test_loss))

  actual_cnt = []
  predicted_cnt = []
  cnt_loss = []
  y_test = y_test.cpu().numpy()

  for i in range(y_test.shape[0]):
    if kernel_type == 'square':
      cnt1 = get_total_cnt_with_redundancy(y_test[i])
      cnt2 = get_total_cnt_with_redundancy(predicted_T[i])
    else:
      cnt1 = get_total_cnt_with_redundancy(y_test[i], f=1)
      cnt2 = get_total_cnt_with_redundancy(predicted_T[i], f=1)
    
    actual_cnt.append(cnt1)
    predicted_cnt.append(cnt2)
    cnt_loss.append(abs(cnt1-cnt2))

    if is_print:
        print(cnt1,cnt2)

  if is_print:
      print('Mean Difference in Counts', np.mean(cnt_loss))

  predicted_cnt = np.array(predicted_cnt)
  predicted_T = np.array(predicted_T)
  return predicted_cnt, predicted_T


CELLS DATASET

In [ ]:
# Training Parameters:
batch_size = 2
epochs = 300
lr = 0.001

prediction_cnt_cells, prediction_T_cells = get_output_count(I_train_cells, T_train_cells, I_val_cells, T_val_cells, I_test_cells, T_test_cells, batch_size=batch_size, epochs=epochs, lr=lr, test_batch_size=2, data_type='cells', kernel_type=cells_kernel)

In [ ]:
im_predicted = prediction_T_cells[31]
temp = im_predicted.reshape(im_predicted.shape[1], im_predicted.shape[2])
im1 = plt.imshow(temp)

In [ ]:
im = T_test_cells[31]
temp2 = im.reshape(im.shape[1], im.shape[2])
im2 = plt.imshow(temp2)

ADIPOCYTE DATASET

In [ ]:
# Training Parameters:
batch_size = 2
epochs = 500
lr = 0.005

prediction_adipocyte, prediction_T_adipocyte = get_output_count(I_train_adipocyte, T_train_adipocyte, I_val_adipocyte, T_val_adipocyte, I_test_adipocyte, T_test_adipocyte, batch_size=batch_size, epochs=epochs, lr=lr, test_batch_size=1, data_type='adipocyte', kernel_type=adipocyte_kernel)

In [ ]:
im_predicted = prediction_T_adipocyte[-16]
temp = im_predicted.reshape(im_predicted.shape[1], im_predicted.shape[2])
im1 = plt.imshow(temp)

In [ ]:
im = T_test_adipocyte[-16]
temp2 = im.reshape(im.shape[1], im.shape[2])
im2 = plt.imshow(temp2)

MBM DATASET

In [ ]:

# Training Parameters:
batch_size = 2
epochs = 400
lr = 0.005

prediction_mbm, prediction_T_mbm = get_output_count(I_train_mbm, T_train_mbm, I_val_mbm, T_val_mbm, I_test_mbm, T_test_mbm, batch_size=batch_size, epochs=epochs, lr=lr, test_batch_size=1, data_type='mbm', kernel_type=mbm_kernel)

In [ ]:
im_predicted = prediction_T_mbm[10]
temp = im_predicted.reshape(im_predicted.shape[1], im_predicted.shape[2])
im1 = plt.imshow(temp)

In [ ]:
im = T_test_mbm[10]
temp2 = im.reshape(im.shape[1], im.shape[2])
im2 = plt.imshow(temp2)

# Convolution Model Keras

In [ ]:
# def conv_model(x_train, y_train, x_val, y_val, x_test, initial_padding=32):
  
#   # input_img = Input(shape = (x_train.shape[1], x_train.shape[2], x_train.shape[3])) #(256,256,3)
#   input_img = Input(shape = (256, 256, 3)) #(256,256,3)
  
#   pad = ZeroPadding2D(initial_padding)(input_img) #(320,320,3)
#   conv1 = Conv2D(64, (3,3), kernel_initializer="glorot_uniform",)(pad)
#   conv1 = LeakyReLU()(conv1)
#   conv1 = BatchNormalization()(conv1) #(318,318,64)

#   conv2a = Conv2D(16, (1,1), kernel_initializer="glorot_uniform",)(conv1)
#   conv2a = LeakyReLU()(conv2a)
#   conv2a = BatchNormalization()(conv2a)
#   conv2b = ZeroPadding2D(1)(conv1)
#   conv2b = Conv2D(16, (3,3), kernel_initializer="glorot_uniform",)(conv2b)
#   conv2b = LeakyReLU()(conv2b)
#   conv2b = BatchNormalization()(conv2b)
#   conv2 = Concatenate()([conv2a, conv2b]) #(318,318,32)

#   conv3a = Conv2D(16, (1,1), kernel_initializer="glorot_uniform",)(conv2)
#   conv3a = LeakyReLU()(conv3a)
#   conv3a = BatchNormalization()(conv3a)
#   conv3b = ZeroPadding2D(1)(conv2)
#   conv3b = Conv2D(32, (3,3), kernel_initializer="glorot_uniform",)(conv3b)
#   conv3b = LeakyReLU()(conv3b)
#   conv3b = BatchNormalization()(conv3b)
#   conv3 = Concatenate()([conv3a, conv3b]) #(318,318,48)

#   conv4 = Conv2D(16, (14,14), kernel_initializer="glorot_uniform",)(conv3)
#   conv4 = LeakyReLU()(conv4)
#   conv4 = BatchNormalization()(conv4) #(305,305,16)

#   conv5a = Conv2D(112, (1,1), kernel_initializer="glorot_uniform",)(conv4)
#   conv5a = LeakyReLU()(conv5a)
#   conv5a = BatchNormalization()(conv5a)
#   conv5b = ZeroPadding2D(1)(conv4)
#   conv5b = Conv2D(48, (3,3), kernel_initializer="glorot_uniform",)(conv5b)
#   conv5b = LeakyReLU()(conv5b)
#   conv5b = BatchNormalization()(conv5b)
#   conv5 = Concatenate()([conv5a, conv5b]) #(305,305,160)

#   conv6a = Conv2D(64, (1,1), kernel_initializer="glorot_uniform",)(conv5)
#   conv6a = LeakyReLU()(conv6a)
#   conv6a = BatchNormalization()(conv6a)
#   conv6b = ZeroPadding2D(1)(conv5)
#   conv6b = Conv2D(32, (3,3), kernel_initializer="glorot_uniform",)(conv6b)
#   conv6b = LeakyReLU()(conv6b)
#   conv6b = BatchNormalization()(conv6b)
#   conv6 = Concatenate()([conv6a, conv6b]) #(305,305,96)

#   conv7a = Conv2D(40, (1,1), kernel_initializer="glorot_uniform",)(conv6)
#   conv7a = LeakyReLU()(conv7a)
#   conv7a = BatchNormalization()(conv7a)
#   conv7b = ZeroPadding2D(1)(conv6)
#   conv7b = Conv2D(40, (3,3), kernel_initializer="glorot_uniform",)(conv7b)
#   conv7b = LeakyReLU()(conv7b)
#   conv7b = BatchNormalization()(conv7b)
#   conv7 = Concatenate()([conv7a, conv7b]) #(305,305,80)

#   conv8a = Conv2D(32, (1,1), kernel_initializer="glorot_uniform",)(conv7)
#   conv8a = LeakyReLU()(conv8a)
#   conv8a = BatchNormalization()(conv8a)
#   conv8b = ZeroPadding2D(1)(conv7)
#   conv8b = Conv2D(96, (3,3), kernel_initializer="glorot_uniform",)(conv8b)
#   conv8b = LeakyReLU()(conv8b)
#   conv8b = BatchNormalization()(conv8b)
#   conv8 = Concatenate()([conv8a, conv8b]) #(305,305,128)

#   conv9 = Conv2D(32, (18,18), kernel_initializer="glorot_uniform",)(conv8)
#   conv9 = LeakyReLU()(conv9)
#   conv9 = BatchNormalization()(conv9) #(288,288,32)

#   conv10 = Conv2D(64, (1,1), kernel_initializer="glorot_uniform",)(conv9)
#   conv10 = LeakyReLU()(conv10)
#   conv10 = BatchNormalization()(conv10) #(288,288,64)

#   conv11 = Conv2D(64, (1,1), kernel_initializer="glorot_uniform",)(conv10)
#   conv11 = LeakyReLU()(conv11)
#   conv11 = BatchNormalization()(conv11) #(288,288,64)

#   conv12 = Conv2D(1, (1,1), kernel_initializer="glorot_uniform",)(conv11)
#   conv12 = LeakyReLU()(conv12)
#   conv12 = BatchNormalization()(conv12) #(288,288,1)

#   # out = Flatten()(conv12)

#   model = models.Model(inputs=input_img, outputs=conv12)
#   opt = optimizers.Adam(0.001)
#   model.compile(loss=tf.keras.losses.MeanAbsoluteError(), optimizer=opt)

#   # model.summary()

#   model.fit(x_train, y_train, batch_size=1, epochs=5, verbose=1, validation_data=(x_val, y_val))

#   # model.summary()

#   # model.fit(x=x_train, y=y_train, batch_size=1, epochs=1, verbose=1, validation_data=(x_val, y_val))
#   # # model.fit(x=x_train, y=y_train, batch_size=1, epochs=1, verbose=1)
#   #   model.compile(loss=tf.keras.losses.MeanAbsoluteError(), optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])

#   y_pred = model.predict(x_test)

#   tf.keras.backend.clear_session()
#   del model

#   return y_pred

In [ ]:
# x_train = []
# for i in range(0,I_train_cells.shape[0],2):
#   I1 = I_train_cells[i]
#   I2 = I_train_cells[i+1]

#   x_train.append((I1,I2))

# x_train = np.array(x_train)
# print(x_train.shape)

# x_val = []
# for i in range(0,I_val_cells.shape[0],2):
#   I1 = I_val_cells[i]
#   I2 = I_val_cells[i+1]

#   x_val.append((I1,I2))

# x_val = np.array(x_val)
# print(x_val.shape)

# x_test = []
# for i in range(0,I_test_cells.shape[0],2):
#   I1 = I_test_cells[i]
#   I2 = I_test_cells[i+1]

#   x_test.append((I1,I2))

# x_test = np.array(x_test)
# print(x_test.shape)


# y_train = []
# for i in range(0,T_train_cells.shape[0],2):
#   T1 = T_train_cells[i]
#   T2 = T_train_cells[i+1]

#   T1 = T1.reshape(T1.shape[0], T1.shape[1], 1)
#   T2 = T2.reshape(T2.shape[0], T2.shape[1], 1)

#   y_train.append((T1,T2))

# y_train = np.array(y_train)
# print(y_train.shape)

# y_val = []
# for i in range(0,T_val_cells.shape[0],2):
#   T1 = T_val_cells[i]
#   T2 = T_val_cells[i+1]

#   T1 = T1.reshape(T1.shape[0], T1.shape[1], 1)
#   T2 = T2.reshape(T2.shape[0], T2.shape[1], 1)

#   y_val.append((T1,T2))

# y_val = np.array(y_val)
# print(y_val.shape)

# y_test = []
# for i in range(0,T_test_cells.shape[0],2):
#   T1 = T_test_cells[i]
#   T2 = T_test_cells[i+1]

#   T1 = T1.reshape(T1.shape[0], T1.shape[1], 1)
#   T2 = T2.reshape(T2.shape[0], T2.shape[1], 1)

#   y_test.append((T1,T2))

# y_test = np.array(y_test)
# print(y_test.shape)

In [ ]:
# T_train_cells = T_train_cells.reshape(T_train_cells.shape[0], T_train_cells.shape[1], T_train_cells.shape[2], 1)
# T_val_cells = T_val_cells.reshape(T_val_cells.shape[0], T_val_cells.shape[1], T_val_cells.shape[2], 1)

# F_I = conv_model(I_train_cells, T_train_cells, I_val_cells, T_val_cells, I_test_cells)

# # F_I = conv_model(x_train, y_train, x_val, y_val, x_test)

In [ ]:
# # print(F_I)
# cnt_loss = []
# for i in range(F_I.shape[0]):
#   # print(get_total_cnt_with_redundancy(F_I[i]))
#   # print(get_total_cnt_with_redundancy(T_test_cells[i]))
#   cnt_loss.append(abs(get_total_cnt_with_redundancy(F_I[i]) - get_total_cnt_with_redundancy(T_test_cells[i])))

# print(np.mean(cnt_loss))

#Keras with 32*32

In [ ]:
# def conv_model_32(x_train, y_train, x_val, y_val, x_test):
  
#   # input_img = Input(shape = (x_train.shape[1], x_train.shape[2], x_train.shape[3])) #(256,256,3)
#   input_img = Input(shape = (32, 32, 3))
  
#   # pad = ZeroPadding2D(initial_padding)(input_img) #(32,32,3)
#   conv1 = Conv2D(64, (3,3), kernel_initializer="glorot_uniform",)(input_img)
#   conv1 = LeakyReLU()(conv1)
#   conv1 = BatchNormalization()(conv1) #(30,30,64)

#   conv2a = Conv2D(16, (1,1), kernel_initializer="glorot_uniform",)(conv1)
#   conv2a = LeakyReLU()(conv2a)
#   conv2a = BatchNormalization()(conv2a)
#   conv2b = ZeroPadding2D(1)(conv1)
#   conv2b = Conv2D(16, (3,3), kernel_initializer="glorot_uniform",)(conv2b)
#   conv2b = LeakyReLU()(conv2b)
#   conv2b = BatchNormalization()(conv2b)
#   conv2 = Concatenate()([conv2a, conv2b]) #(30,30,32)

#   conv3a = Conv2D(16, (1,1), kernel_initializer="glorot_uniform",)(conv2)
#   conv3a = LeakyReLU()(conv3a)
#   conv3a = BatchNormalization()(conv3a)
#   conv3b = ZeroPadding2D(1)(conv2)
#   conv3b = Conv2D(32, (3,3), kernel_initializer="glorot_uniform",)(conv3b)
#   conv3b = LeakyReLU()(conv3b)
#   conv3b = BatchNormalization()(conv3b)
#   conv3 = Concatenate()([conv3a, conv3b]) #(30,30,48)

#   conv4 = Conv2D(16, (14,14), kernel_initializer="glorot_uniform",)(conv3)
#   conv4 = LeakyReLU()(conv4)
#   conv4 = BatchNormalization()(conv4) #(17,17,16)

#   conv5a = Conv2D(112, (1,1), kernel_initializer="glorot_uniform",)(conv4)
#   conv5a = LeakyReLU()(conv5a)
#   conv5a = BatchNormalization()(conv5a)
#   conv5b = ZeroPadding2D(1)(conv4)
#   conv5b = Conv2D(48, (3,3), kernel_initializer="glorot_uniform",)(conv5b)
#   conv5b = LeakyReLU()(conv5b)
#   conv5b = BatchNormalization()(conv5b)
#   conv5 = Concatenate()([conv5a, conv5b]) #(17,17,160)

#   conv6a = Conv2D(64, (1,1), kernel_initializer="glorot_uniform",)(conv5)
#   conv6a = LeakyReLU()(conv6a)
#   conv6a = BatchNormalization()(conv6a)
#   conv6b = ZeroPadding2D(1)(conv5)
#   conv6b = Conv2D(32, (3,3), kernel_initializer="glorot_uniform",)(conv6b)
#   conv6b = LeakyReLU()(conv6b)
#   conv6b = BatchNormalization()(conv6b)
#   conv6 = Concatenate()([conv6a, conv6b]) #(17,17,96)

#   conv7a = Conv2D(40, (1,1), kernel_initializer="glorot_uniform",)(conv6)
#   conv7a = LeakyReLU()(conv7a)
#   conv7a = BatchNormalization()(conv7a)
#   conv7b = ZeroPadding2D(1)(conv6)
#   conv7b = Conv2D(40, (3,3), kernel_initializer="glorot_uniform",)(conv7b)
#   conv7b = LeakyReLU()(conv7b)
#   conv7b = BatchNormalization()(conv7b)
#   conv7 = Concatenate()([conv7a, conv7b]) #(17,17,80)

#   conv8a = Conv2D(32, (1,1), kernel_initializer="glorot_uniform",)(conv7)
#   conv8a = LeakyReLU()(conv8a)
#   conv8a = BatchNormalization()(conv8a)
#   conv8b = ZeroPadding2D(1)(conv7)
#   conv8b = Conv2D(96, (3,3), kernel_initializer="glorot_uniform",)(conv8b)
#   conv8b = LeakyReLU()(conv8b)
#   conv8b = BatchNormalization()(conv8b)
#   conv8 = Concatenate()([conv8a, conv8b]) #(17,17,128)

#   conv9 = Conv2D(32, (17,17), kernel_initializer="glorot_uniform",)(conv8)
#   conv9 = LeakyReLU()(conv9)
#   conv9 = BatchNormalization()(conv9) #(1,1,32)

#   conv10 = Conv2D(64, (1,1), kernel_initializer="glorot_uniform",)(conv9)
#   conv10 = LeakyReLU()(conv10)
#   conv10 = BatchNormalization()(conv10) #(1,1,64)

#   conv11 = Conv2D(64, (1,1), kernel_initializer="glorot_uniform",)(conv10)
#   conv11 = LeakyReLU()(conv11)
#   conv11 = BatchNormalization()(conv11) #(1,1,64)

#   conv12 = Conv2D(1, (1,1), kernel_initializer="glorot_uniform",)(conv11)
#   conv12 = LeakyReLU()(conv12)
#   conv12 = BatchNormalization()(conv12) #(1,1,1)

#   # out = Flatten()(conv12)

#   model = models.Model(inputs=input_img, outputs=conv12)
#   opt = optimizers.Adam(0.005)
#   model.compile(loss=tf.keras.losses.MeanAbsoluteError(), optimizer=opt)

#   # model.summary()

#   model.fit(x_train, y_train, batch_size=4, epochs=100, verbose=1, validation_data=(x_val, y_val))

#   # model.summary()

#   # model.fit(x=x_train, y=y_train, batch_size=1, epochs=1, verbose=1, validation_data=(x_val, y_val))
#   # # model.fit(x=x_train, y=y_train, batch_size=1, epochs=1, verbose=1)
#   #   model.compile(loss=tf.keras.losses.MeanAbsoluteError(), optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])

#   y_pred = model.predict(x_test)

#   tf.keras.backend.clear_session()
#   del model

#   return y_pred

In [ ]:
# I_train_cells_32 = zero_pad_3d(I_train_cells, 32)
# I_val_cells_32 = zero_pad_3d(I_val_cells, 32)
# I_test_cells_32 = zero_pad_3d(I_test_cells, 32)

# L_train_cells_32 = zero_pad(L_train_cells,32)
# L_val_cells_32 = zero_pad(L_val_cells,32)
# L_test_cells_32 = zero_pad(L_test_cells,32)

In [ ]:
# def conv_to_32(L, stride=1, f=32, is_y=False):

#     # Retrieve dimensions from L's shape (≈1 line)  
#     # (m, n_H_prev, n_W_prev, n_D_prev) = (L.shape[0] , L.shape[1] , L.shape[2], L.shape[3])
    
#     # Compute the dimensions of the CONV output volume using the formula given above. Hint: use int() to floor. (≈2 lines)
#     # n_H = int( (n_H_prev - f + 1)/stride ) 
#     # n_W = int( (n_W_prev - f + 1)/stride )

#     # n_H = int(n_H_prev/f)
#     # n_W = int(n_W_prev/f)
    
#     # Initialize the output volume Z with zeros. (≈1 line)
#     if is_y:
#       (m, n_H_prev, n_W_prev) = (L.shape[0] , L.shape[1] , L.shape[2])
#       n_H = int(n_H_prev/f)
#       n_W = int(n_W_prev/f)
#       Z = np.zeros([m*n_H*n_W,1])
#     else:
#       (m, n_H_prev, n_W_prev, n_D_prev) = (L.shape[0] , L.shape[1] , L.shape[2], L.shape[3])
#       n_H = int(n_H_prev/f)
#       n_W = int(n_W_prev/f)
#       Z = np.zeros([m*n_H*n_W, f, f, n_D_prev])
#     # Z = []
    
#     # Create L_pad by padding L
#     # L_pad = zero_pad(L,pad)
#     cnt = 0
    
#     for i in range(m):                               # loop over the batch of training examples
#         a_prev = L[i]                               # Select ith training example's padded activation
#         for h in range(n_H):                           # loop over vertical axis of the output volume
#             for w in range(n_W):                       # loop over horizontal axis of the output volume
#                 # for c in range(n_C):                   # loop over channels (= #filters) of the output volume
                    
#                     # Find the corners of the current "slice" (≈4 lines)
#                     vert_start = h*f
#                     vert_end = h*f + f
#                     horiz_start = w*f
#                     horiz_end = w*f + f
                    
#                     # Use the corners to define the (3D) slice of a_prev_pad (See Hint above the cell). (≈1 line)
#                     # a_slice_prev = a_prev[vert_start:vert_end,horiz_start:horiz_end,:]
                    
#                     # Convolve the (3D) slice with the correct filter W and bias b, to get back one output neuron. (≈1 line)
#                     if is_y:
#                       a_slice_prev = a_prev[vert_start:vert_end,horiz_start:horiz_end]
#                       Z[cnt] = get_total_cnt_no_redundancy(a_slice_prev)
#                     else:
#                       a_slice_prev = a_prev[vert_start:vert_end,horiz_start:horiz_end,:]
#                       Z[cnt] = a_slice_prev
#                     cnt += 1
#                     # Z.append(a_slice_prev)

    
#     # Making sure your output shape is correct
#     # assert(Z.shape == (m, n_H, n_W))
    
#     # Save information in "cache" for the backprop
#     # cache = (A_prev, W, b, hparameters)
    
#     # Z = np.array(Z)
#     # assert(Z.shape == (m*n_H*n_W, 32, 32, n_D_prev))
#     return Z

In [ ]:
# x_train_32 = conv_to_32(I_train_cells_32)
# x_val_32 = conv_to_32(I_val_cells_32)
# x_test_32 = conv_to_32(I_test_cells_32)

In [ ]:
# y_train_32 = conv_to_32(L_train_cells_32, is_y=True)
# y_val_32 = conv_to_32(L_val_cells_32, is_y=True)
# y_test_32 = conv_to_32(L_test_cells_32, is_y=True)

# y_train_32 = y_train_32.reshape(y_train_32.shape[0],1,1)
# y_val_32 = y_val_32.reshape(y_val_32.shape[0],1,1)
# y_test_32 = y_test_32.reshape(y_test_32.shape[0],1,1)

In [ ]:
# print(x_train_32.shape)
# print(y_train_32.shape)

In [ ]:
# x = conv_model_32(x_train_32, y_train_32, x_val_32, y_val_32, x_test_32)

In [ ]:
# for i in range(x_test_32.shape[0]):
#   print(x[i], y_test_32[i])